# ICD-10 codes CSV file code extraction
<hr>
When conducting a disease query, one may want to find all cases related to certain disease state such as chronic inflammation. However, it may be difficult to discover all ICD-10 diagnosis codes by hand from literature. Therefore I created this simple python notebook, that queries ICD-10 codes with keywords from a table provided by <a href="https://github.com/kamillamagna/ICD-10-CSV">kamillamagna</a>.

In [1]:
import pandas as pd
import numpy as np
import re

titles = ['Category Code','Diagnosis Code','Full Code','Abbreviated Description','Full Description','Category Title']
df = pd.read_csv("https://raw.githubusercontent.com/kamillamagna/ICD-10-CSV/master/codes.csv", names=titles)

In [2]:
df.head()

,Category Code,Diagnosis Code,Full Code,Abbreviated Description,Full Description,Category Title
0,A00,0,A000,"Cholera due to Vibrio cholerae 01, biovar chol...","Cholera due to Vibrio cholerae 01, biovar chol...",Cholera
1,A00,1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor","Cholera due to Vibrio cholerae 01, biovar eltor",Cholera
2,A00,9,A009,"Cholera, unspecified","Cholera, unspecified",Cholera
3,A010,0,A0100,"Typhoid fever, unspecified","Typhoid fever, unspecified",Typhoid fever
4,A010,1,A0101,Typhoid meningitis,Typhoid meningitis,Typhoid fever


In [3]:
def ICDquery(df, inclusion, exclusion, search_name):
    dfquery1 = df[(df["Abbreviated Description"].str.contains(inclusion, flags=re.IGNORECASE, regex=True)) & (~df["Abbreviated Description"].str.contains(exclusion, flags=re.IGNORECASE, regex=True))]
    dfquery2 = df[(df["Full Description"].str.contains(inclusion, flags=re.IGNORECASE, regex=True)) & (~df["Full Description"].str.contains(exclusion, flags=re.IGNORECASE, regex=True))]
    dfquery3 = df[(df["Category Title"].str.contains(inclusion, flags=re.IGNORECASE, regex=True)) & (~df["Category Title"].str.contains(exclusion, flags=re.IGNORECASE, regex=True))]

    dfquery = pd.concat([dfquery1, dfquery2, dfquery3]).drop_duplicates(subset="Full Code")
    print("Initial ICD-10 codes: " + str(len(df)))
    print("Initial ICD-10 category codes: " + str(len(df.drop_duplicates(subset="Category Code"))))
    print("ICD-10 codes concerning " + search_name + ": " + str(len(dfquery)))
    print("ICD-10 category codes concerning  " + search_name + ": " + str(len(dfquery.drop_duplicates(subset="Category Code"))))
    return(dfquery)

### Subset query

In [4]:
# query function arguments: dataframe to be queried, query keywords, name of the query
dfitis = ICDquery(df, "itis|inflam", "noninflam|non-inflam", "inflammation")

Initial ICD-10 codes: 71704
Initial ICD-10 category codes: 19927
ICD-10 codes concerning inflammation: 3396
ICD-10 category codes concerning  inflammation: 1218


### Query of a subset from the initial subset

In [5]:
dfchronic = ICDquery(dfitis, "subacute|chronic", "justaplaceholderstring", "subacute/chronic inflammation")

Initial ICD-10 codes: 3396
Initial ICD-10 category codes: 1218
ICD-10 codes concerning subacute/chronic inflammation: 369
ICD-10 category codes concerning  subacute/chronic inflammation: 137


In [8]:
# for some reason, the separator is missing from the codes, so I'll add it with this function
def addseparator(series):
    for i in range(len(series)):
        if "." not in series.values[i]:
            if len(series.values[i]) > 3:
                series.values[i] = (str(series.values[i][:3]) + "." + str(series.values[i][3:]))
    return(series)

In [9]:
dfitis["Full Code"] = addseparator(dfitis["Full Code"])
dfitis["Category Code"] = addseparator(dfitis["Category Code"])
dfchronic["Full Code"] = addseparator(dfchronic["Full Code"])
dfchronic["Category Code"] = addseparator(dfchronic["Category Code"])

### Get aggregate data from queried ICD-10 codes

In [10]:
def get_category_aggregate(df):
    lista = []
    for i in range(len(df)):    
        lista.append(df.iloc[i,0][:3])
    return(dict((x,lista.count(x)) for x in set(lista)))

def get_category_aggregate2(df):
    lista = []
    for i in range(len(df)):    
        lista.append(df.iloc[i,0][:2])
    return(dict((x,lista.count(x)) for x in set(lista)))

In [11]:
category_code_agg = get_category_aggregate(dfchronic)
dfagg1 = pd.DataFrame.from_dict(category_code_agg, orient='index',columns=["Number"]).sort_values(by=['Number'],ascending=False)
print(dfagg1.shape)
dfagg1

(56, 1)


,Number
M86,120
H65,24
K51,21
H10,17
K80,16
H66,12
H04,12
K05,11
I87,8
H70,8


In [12]:
category_code_agg2 = get_category_aggregate2(dfchronic)
dfagg2 = pd.DataFrame.from_dict(category_code_agg2, orient='index',columns=["Number"]).sort_values(by=['Number'],ascending=False)
print(dfagg2.shape)
dfagg2

(34, 1)


,Number
M8,120
H6,48
K5,21
K8,20
H1,17
J3,15
H0,13
H7,12
K0,12
I8,9


### Reference sheet

Chapter|Code Range|Description
---|---|---
1|A00-B99|Certain Infectious and Parasitic Diseases
2|C00-D49|Neoplasms
3|D50-D89|Diseases of the Blood and Blood-Forming Organs and Certain Disorders Involving the Immune Mechanism
4|E00-E89|Endocrine, Nutritional and Metabolic Diseases
5|F01-F99|Mental, Behavioral and Neurodevelopmental Disorders
6|G00-G99|Diseases of the Nervous System
7|H00-H59|Diseases of the Eye and Adnexa
8|H60-H95|Diseases of the Ear and Mastoid Process
9|I00-I99|Diseases of the Circulatory System
10|J00-J99|Diseases of the Respiratory System
11|K00-K95|Diseases of the Digestive System
12|L00-L99|Diseases of the Skin and Subcutaneous Tissue
13|M00-M99|Diseases of the Musculoskeletal System and Connective Tissue
14|N00-N99|Diseases of the Genitourinary System
15|O00-O9A|Pregnancy, Childbirth and the Puerperium
16|P00-P96|Certain Conditions Originating in the Perinatal Period
17|Q00-Q99|Congenital Malformations, Deformations, and Chromosomal Abnormalities
18|R00-R99|Symptoms, Signs, and Abnormal Clinical and Laboratory Findings, Not Elsewhere Classified
19|S00-T88|Injury, Poisoning, and Certain Other Consequences of External Causes
20|V00-Y99|External Causes of Morbidity
21|Z00-Z99|Factors Influencing Health Status and Contact with Health Services

## Export to CSV

In [13]:
dfitis.set_index("Category Code").to_csv("ICD10Inflammation2.csv")
dfchronic.set_index("Category Code").to_csv("ICD10ChronicInflammation2.csv")